In [30]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
xp = cuda.cupy if gpuid >= 0 else np

In [32]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [44]:
model = SpeechEncoderDecoder(SPEECH_DIM, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)
if gpuid >= 0:
    cuda.get_device(gpuid).use()
    model.to_gpu()

In [5]:
log_train_fil_name, text_fname, dev_fname, test_fname

('es_speech_to_en_char_better/train_17394sen_4-2layers_512units_callhome_callhome_es_en_1.log',
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/train.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_train.es'},
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/dev.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_dev.es'},
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/test.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_test.es'})

In [6]:
%aimport nmt_trials
from nmt_trials import *

Starting experiment
num sentences=13137 and num epochs=0


In [7]:
buckets, bucket_lengths = populate_buckets()

Splitting data into 34 buckets, each of width=24


 19%|█▉        | 2519/13137 [00:07<00:27, 381.18it/s]

file=053.181 has a nan value, fr len=1, en len=3


100%|██████████| 13137/13137 [00:36<00:00, 359.99it/s]


Saving bucket data


In [8]:
25*32, 24*35

(800, 840)

In [9]:
_, en, sf = get_data_item('053.181')
xp.isnan(xp.sum(sf))

array(True, dtype=bool)

In [45]:
buck_indx = 23
i = 0
batch_size = 50

pad_size_speech = (buck_indx+1) * SPEECH_BUCKET_WIDTH
pad_size_en = min(bucket_lengths[buck_indx][3], MAX_EN_LEN)

src_lim = pad_size_speech
tar_lim = pad_size_en

sp_files_in_batch = [t[0] for t in buckets[buck_indx][i:i+batch_size]]

# get the next batch of data
batch_data = []
for sp_fil in sp_files_in_batch:
    _, en_ids, speech_feat = get_data_item(sp_fil, cat="train")
    # print(speech_feat.shape, len(en_ids))
    batch_data.append((speech_feat[:pad_size_speech], en_ids[:pad_size_en]))

# compute loss
# loss = model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=True)

In [46]:
batch_size = len(batch_data)
in_shape_r, in_shape_c = batch_data[0][0].shape

fwd_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
rev_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
decoder_batch = xp.full((batch_size, pad_size_en+2), PAD_ID, dtype=xp.int32)

In [47]:
for i, (src, tar) in enumerate(batch_data):
    fwd_encoder_batch[i] = model.pad_array(src, src_lim)
    rev_encoder_batch[i] = model.pad_array(xp.flip(src, axis=0), src_lim)

    tar_data = [GO_ID] + tar + [EOS_ID]
    decoder_batch[i] = model.pad_list(tar_data, tar_lim+2, at_start=False)

In [51]:
train=True

In [48]:
fwd_encoder_batch = xp.swapaxes(fwd_encoder_batch, 0,1)
rev_encoder_batch = xp.swapaxes(rev_encoder_batch, 0,1)
decoder_batch = xp.swapaxes(decoder_batch, 0,1)

In [49]:
L_FWD_STATES = model.encode_speech_batch_lstm(fwd_encoder_batch, model.lstm_enc, train=True)

In [52]:
L_REV_STATES = model.encode_speech_batch_lstm(rev_encoder_batch, model.lstm_rev_enc, train)
# reverse the states to align them with forward encoder
L_REV_STATES = xp.flip(L_REV_STATES, axis=0)

In [54]:
return_shape = L_FWD_STATES.shape
model.enc_states = F.concat((L_FWD_STATES, L_REV_STATES), axis=2)
model.enc_states = F.swapaxes(model.enc_states, 0, 1)

In [55]:
model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=train)

In [ ]:
L_FWD_STATES.shape

In [ ]:
pad_size_speech / 8

In [ ]:
in_size, batch_size, in_dim = fwd_encoder_batch.shape

In [ ]:
print(in_size, batch_size, in_dim)